Используйте версию библиотеки scikit-learn==0.23.0, также Вам потребуется библиотека opencv

In [1]:
!pip install --upgrade pip
!pip install imutils
!pip install opencv-python
!pip install --upgrade scikit-learn==0.23.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 29.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the follo

In [9]:
import cv2
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import path
from imutils import paths

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

from sklearn.metrics import accuracy_score

Гистограммы — характеристики распределения интенсивности изображений

In [3]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

In [15]:
imagePaths = sorted(list(paths.list_images('/content/drive/MyDrive/Colab Notebooks/train')))
data = []

for imagePath in imagePaths:
  image_hist = list(extract_histogram(cv2.imread(imagePath)))
  if "/cat." in imagePath:
    image_hist.append(1)
  else:
    image_hist.append(0)
  data.append(np.array(image_hist))

In [17]:
df = pd.DataFrame(data, columns=[str(i) for i in range(len(data[0]))])
df.rename(columns={'512': 'y'}, inplace=True)
print(df)
X = df.loc[:, '0':'511']
y = df.loc[:, 'y']

            0         1         2         3         4    5    6    7  \
0    0.242645  0.073122  0.000232  0.000000  0.000000  0.0  0.0  0.0   
1    0.418818  0.126781  0.000085  0.000000  0.000000  0.0  0.0  0.0   
2    0.356737  0.184168  0.020616  0.001619  0.000717  0.0  0.0  0.0   
3    0.001266  0.019995  0.011941  0.001583  0.000158  0.0  0.0  0.0   
4    0.000000  0.000105  0.000000  0.000000  0.000000  0.0  0.0  0.0   
..        ...       ...       ...       ...       ...  ...  ...  ...   
995  0.134753  0.010071  0.000000  0.000000  0.000000  0.0  0.0  0.0   
996  0.122335  0.165961  0.041414  0.016499  0.000102  0.0  0.0  0.0   
997  0.081336  0.033835  0.000000  0.000000  0.000000  0.0  0.0  0.0   
998  0.598071  0.090467  0.006297  0.000019  0.000000  0.0  0.0  0.0   
999  0.501765  0.108061  0.017979  0.003817  0.000026  0.0  0.0  0.0   

            8         9  ...       503  504       505     506      507  \
0    0.000000  0.087363  ...  0.000000  0.0  0.000000  0.0000

In [18]:
linear_svc = LinearSVC(C = 1.09, random_state = 462)
bagging_clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 462), n_estimators = 19, random_state = 462)
random_forest = RandomForestClassifier(n_estimators = 19, criterion = 'entropy', min_samples_leaf = 10, max_leaf_nodes = 20, random_state = 462)

final_estimator = LogisticRegression(solver='lbfgs', random_state = 462)

estimators = [('linear_svc', linear_svc),
              ('bagging_clf', bagging_clf),
              ('random_forest', random_forest)]
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator, cv=2)
clf.fit(X, y)

y_pred = clf.predict(X)
print('доля правильной классификации', np.round(accuracy_score(y, y_pred), 2))

доля правильной классификации 0.84


In [19]:
imageTests = sorted(list(paths.list_images('/content/drive/MyDrive/Colab Notebooks/test')))

for imageTest in imageTests:
  if 'dog.1023.jpg' in imageTest:
    dog_1023 = extract_histogram(cv2.imread(imageTest))
  if 'dog.1029.jpg' in imageTest:
    dog_1029 = extract_histogram(cv2.imread(imageTest))
  if 'dog.1006.jpg' in imageTest:
    dog_1006 = extract_histogram(cv2.imread(imageTest))
  if 'cat.1004.jpg' in imageTest:
    cat_1004 = extract_histogram(cv2.imread(imageTest))
    
print(np.round(*clf.predict_proba(dog_1023.reshape(1, -1)), 3)[1])
print(np.round(*clf.predict_proba(dog_1029.reshape(1, -1)), 3)[1])
print(np.round(*clf.predict_proba(dog_1006.reshape(1, -1)), 3)[1])
print(np.round(*clf.predict_proba(cat_1004.reshape(1, -1)), 3)[1])

0.351
0.272
0.376
0.416
